## Modules

In [ ]:
import os
import numpy as np
import pandas as pd
import ast

from multiprocessing import Pool, cpu_count
from tqdm.notebook import tqdm

from GridPythonModule import *
from GPM_wrapper import *
from knot_floer_homology import *

# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

## Functions

In [ ]:
def get_fitted_grid(G):
    rtn = None
    G = simplify_grid(G)
    cr = crossing_number(G)
    nc = number_of_components(G)
    if nc == 1 and cr > 16:
        rtn = (G,cr)
    return rtn
#---    
def calc_hfk(G_tuple):
    G, cr = G_tuple
    rtn = None
    PD = Grid(*G).get_PD()
    hfk = pd_to_hfk(PD)
    if hfk['fibered']:
        sg = hfk['seifert_genus']
        ranks = hfk['ranks']
        hfk['lmd'] = []
        for pg in ranks.keys():
            if pg[0] == sg:
                hfk['lmd'].append(pg[1])
            elif pg[0] == (-1)*sg:
                hfk['lmd'].append((-1)*pg[1])
        rtn = {'cr_num': cr, 'seifert_genus': sg, 'lmd': hfk['lmd'], 'ranks': hfk['ranks'], 'grid': G, 'PD notation': PD}        
    return rtn

## Scratch

In [ ]:
result = []
N, gsize = 50, 20

with tqdm(total=N, desc = '# of required knots') as pbar:
    while len(result) < N:
        ### Collect fitted grids
        Glist, Glist_len = [], 80
        with tqdm(total=Glist_len, desc = 'generating Glist') as pbar_Glist:
            fails, fmax = 0, 100
            while len(Glist) < Glist_len and fails < fmax:
                with Pool(processes=8) as pool:
                    grids = [generate_random_grid(gsize) for _ in range(100)]
                    output = [v for v in pool.map(get_fitted_grid, grids) if v is not None]
                    if len(output) > 0: 
                        Glist += output
                        pbar_Glist.update(len(output))
                    else:
                        fails += 1
            if fails >= fmax:
                print(f"Fail to generate Glist {fails} times!")
                break

        ### Collect knots satisfying seifert genus > lamda
        with Pool(processes=8) as pool:
            output = [v for v in list(tqdm(pool.imap(calc_hfk, Glist), total = len(Glist), desc='hfk calc')) if v is not None]
            if len(output) > 0:
                result += output
                pbar.update(len(output))
#---
DF = pd.DataFrame(result); display(DF)
filename = f"./data/hfk_for_knots_with_over_16_crossings.csv"
DF.to_csv(filename, mode = 'a', index=False, header = (not os.path.isfile(filename)))

In [ ]:
count, trial_Max = 0, 10**3 
gsize, Glist_len = 30, 24

with tqdm(total=trial_Max, desc = 'Trial Count') as pbar:
    result = []
    while (not result) and count < trial_Max:
        ### Collect fitted grids
        Glist = []
        fails, fmax = 0, 100
        while len(Glist) < Glist_len and fails < fmax:
            with Pool(processes=8) as pool:
                grids = [generate_random_grid(gsize) for _ in range(100)]
                output = [v for v in pool.map(get_fitted_grid, grids) if v is not None]
                if len(output) > 0: 
                    Glist += output
                else:
                    fails += 1
        if fails >= fmax:
            print(f"Fail to generate Glist {fails} times!")
            break
        else:
            print(f"\rGlist has been ready. len(Glist)={len(Glist)}", end="")

        ### Collect knots with negative lamda
        with Pool(processes=8) as pool:
            output = [v for v in pool.imap(calc_hfk, Glist) if v is not None]
            if len(output) > 0:
                for hfk in output:
                    if sum(x<0 for x in hfk['lmd']) > 0:
                        result.append(hfk)
        count += 1
        pbar.update(count)
        print(f"\rCompleted the computation for hfk.", end="")
#---
if result:
    DF = pd.DataFrame(result); display(DF)
    # filename = f"./data/knots_with_negative_lambda.csv"
    # DF.to_csv(filename, mode = 'a', index=False, header = (not os.path.isfile(filename)))